In [1]:
import tensorflow as tf
import torch as tc
import numpy as np
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from torch.autograd import grad as tcgrad
DTYPE = tf.float64

In [2]:
def convert_to_torch(tensors):
    new_tensors = []
    for tensor in tensors:
        new_tensor = tc.tensor(tensor.numpy(), requires_grad=True)
        new_tensors.append(new_tensor)
    return new_tensors

def fn(A, B, W):
    return tc.log(tc.abs(tc.sum(W * A.det().view(-1,n_k,1,1) * B.det().view(-1,n_k,1,1), axis=1, keepdim=True)))

def compute_ae_vectors(r_atoms, r_electrons):
    # ae_vectors (n_samples, n_electrons, n_atoms, 3)
    r_atoms = tf.expand_dims(r_atoms, 1)
    r_electrons = tf.expand_dims(r_electrons, 2)
    ae_vectors = r_electrons - r_atoms
    return ae_vectors

def compute_relative_vectors(v1, v2):
    relative_vectors = v1.unsqueeze(2) - v2.unsqueeze(1)
    return relative_vectors

def compare_tensors(ptorch, tflow):
    ptorch = ptorch.detach().numpy()
    tflow = tflow.numpy()
    mask = np.isclose(ptorch, tflow, rtol=0.0, atol=1e-4).astype(np.float32)
    return np.mean(mask)

def compare_tf_tensors(tflow1, tflow2,rtol=1e-05, atol=1e-08): #
    tflow1 = tflow1.numpy()
    tflow2 = tflow2.numpy()
    mask = np.isclose(tflow1, tflow2).astype(np.float32)
    return np.mean(mask) 

ct = compare_tensors

In [3]:

def slogdet_keepdim(tensor):
    sign, tensor_out = tf.linalg.slogdet(tensor)
    tensor_out = tf.reshape(tensor_out, (*tensor_out.shape, 1, 1))
    sign = tf.reshape(sign, (*sign.shape, 1, 1))
    return sign, tensor_out


def generate_gamma(s):
    n_egvs = s.shape[2]
    gamma = [tf.reduce_prod(s[:, :, :i], axis=-1) * tf.reduce_prod(s[:, :, i+1:], axis=-1) for i in range(n_egvs-1)]
    gamma.append(tf.reduce_prod(s[:, :, :-1], axis=-1))
    gamma = tf.stack(gamma, axis=2)
    gamma = tf.expand_dims(gamma, axis=2)
    return gamma


def first_derivative_det(A):
    with tf.device("/cpu:0"):  # this is incredible stupid /// its actually not
        s, u, v = tf.linalg.svd(A, full_matrices=False)
    # s, u, v = tf.linalg.svd(A, full_matrices=False)
    v_t = tf.linalg.matrix_transpose(v)
    gamma = generate_gamma(s)
    sign = (tf.linalg.det(u) * tf.linalg.det(v))[..., None, None]
    out = sign * ((u * gamma) @ v_t)
    return out, (s, u, v_t, sign)


def generate_p(s):
    """
    :param s:
    :return:
    """
    n_samples, n_k, n_dim = s.shape
    new_shape = (1, 1, 1, n_dim, n_dim)
    s = s[..., None, None]
    s = tf.tile(s, new_shape)
    mask = np.ones(s.shape, dtype=np.bool)
    for i in range(n_dim):
        for j in range(n_dim):
            mask[..., i, i, j] = False
            mask[..., j, i, j] = False
    mask = tf.convert_to_tensor(mask)
    s = tf.where(mask, s, tf.ones_like(s, dtype=s.dtype))
    s_prod = tf.reduce_prod(s, axis=-3)
    s_prod = tf.linalg.set_diag(s_prod, tf.zeros((s_prod.shape[:-1]), dtype=s.dtype))
    return s_prod


def second_derivative_det(A, C_dash, *A_cache):
    """

    :param A:
    :param C_dash:
    :param A_cache:
    :return:
    """
    # This function computes the second order derivative of detA wrt to A
    # A matrix
    # C_bar backward sensitivity
    # A_cache cached values returned by grad_det(A)
    s, u, v_t, sign = A_cache  # decompose the cache

    M = v_t @ tf.linalg.matrix_transpose(C_dash) @ u

    p = generate_p(s)

    sgn = tf.math.sign(sign)

    m_jj = tf.linalg.diag_part(M)
    xi = -M * p

    xi_diag = p @ tf.expand_dims(m_jj, -1)
    xi = tf.linalg.set_diag(xi, tf.squeeze(xi_diag, -1))
    return sgn * u @ xi @ v_t


def k_sum(x):
    return tf.reduce_sum(x, axis=1, keepdims=True)


def matrix_sum(x):
    return tf.reduce_sum(x, axis=[-2, -1], keepdims=True)


@tf.function
def _log_abs_sum_det_fwd(a, b, w):
    a = tf.stop_gradient(a)
    b = tf.stop_gradient(b)
    w = tf.stop_gradient(w)
    """

    :param a:
    :param b:
    :param w:
    :return:
    """
    # Take the slogdet of all k determinants
    sign_a, logdet_a = slogdet_keepdim(a)
    sign_b, logdet_b = slogdet_keepdim(b)

    x = logdet_a + logdet_b
    xmax = tf.math.reduce_max(x, axis=1, keepdims=True)

    unshifted_exp = sign_a * sign_b * tf.exp(x)
    unshifted_exp_w = w * unshifted_exp
    sign_unshifted_sum = tf.math.sign(tf.reduce_sum(unshifted_exp_w, axis=1, keepdims=True))

    exponent = x - xmax
    shifted_exp = sign_a * sign_b * tf.exp(exponent)

    u = w * shifted_exp
    u_sum = tf.reduce_sum(u, axis=1, keepdims=True)
    sign_shifted_sum = tf.math.sign(u_sum)
    log_psi = tf.math.log(tf.math.abs(u_sum)) + xmax

    # Both of these derivations appear to be valid
    # activations = shifted_exp
    # sensitivities = sign_unshifted_sum * tf.exp(-log_psi)
    # dw = sign_unshifted_sum * sign_a * sign_b * tf.exp(x-log_psi)
    #
    # return log_psi, sign_shifted_sum, activations, sensitivities, \
    #           (a, b, w, unshifted_exp, sign_unshifted_sum, dw, sign_a, logdet_a, sign_b, logdet_b, log_psi)

    sensitivities = tf.exp(-log_psi) * sign_unshifted_sum
    # sensitivities = tf.exp(xmax-log_psi) * sign_shifted_sum

    dw = sign_unshifted_sum * sign_a * sign_b * tf.exp(x - log_psi)

    return log_psi, sign_unshifted_sum, unshifted_exp, sensitivities, \
           (a, b, w, unshifted_exp, sign_unshifted_sum, dw, sign_a, logdet_a, sign_b, logdet_b, log_psi)

# @tf.function
def _log_abs_sum_det_first_order(*fwd_cache):
    """

    :param fwd_cache:
    :return:
    """
    a, b, w, unshifted_exp, sign_unshifted_sum, dw, sign_a, logdet_a, sign_b, logdet_b, log_psi = fwd_cache

    ddeta, ddeta_cache = first_derivative_det(a)
    ddetb, ddetb_cache = first_derivative_det(b)

    dfddeta = w * sign_unshifted_sum * sign_b * tf.exp(logdet_b - log_psi)
    dfddetb = w * sign_unshifted_sum * sign_a * tf.exp(logdet_a - log_psi)

    da = dfddeta * ddeta
    db = dfddetb * ddetb

    return (da, db, dw), (sign_unshifted_sum, ddeta, ddeta_cache, ddetb, ddetb_cache, dfddeta, dfddetb, da, db, dw)


# @tf.function
def _log_abs_sum_det_second_order(a_dash, b_dash, w_dash, *cache):
    """

    :param a_dash:
    :param b_dash:
    :param w_dash:
    :param cache:
    :return:
    """
    a, b, w, unshifted_exp, sign_unshifted_sum, _, sign_a, logdet_a, sign_b, logdet_b, log_psi, \
    sign_u, ddeta, ddeta_cache, ddetb, ddetb_cache, dfddeta, dfddetb, da, db, dw = cache

    dfddeta_w = dfddeta / w
    dfddetb_w = dfddetb / w

    ddeta_sum = matrix_sum(a_dash * ddeta)
    da_sum = matrix_sum(da * a_dash)
    ddetb_sum = matrix_sum(b_dash * ddetb)
    db_sum = matrix_sum(db * b_dash)
    a_sum = k_sum(dfddeta * ddeta_sum)
    b_sum = k_sum(dfddetb * ddetb_sum)

    # Compute second deriviate of f wrt to w
    d2w = w_dash * -dw * k_sum(dw)

    # compute deriviate of df/da wrt to w
    dadw = -dw * k_sum(da_sum)
    dadw += dfddeta_w * ddeta_sum  # i=j

    # compute derivative of df/db wrt to w
    dbdw = -dw * k_sum(db_sum)
    dbdw += dfddetb_w * ddetb_sum  # i=j

    # Compute second derivative of f wrt to a
    d2a = -da * a_sum
    d2a += dfddeta * second_derivative_det(a, a_dash, *ddeta_cache)  # i=j
    # Compute derivative of df/db wrt to a
    dbda = -da * b_sum
    dbda += ddeta * sign_u * tf.exp(-log_psi) * w * ddetb_sum  # i=j
    # Compute derivative of df/dw wrt to a
    dwda = w_dash * -da * k_sum(dw)
    dwda += w_dash * da / w  # i=j

    # Compute second derivative of f wrt to b
    d2b = -db * b_sum
    d2b += dfddetb * second_derivative_det(b, b_dash, *ddetb_cache)  # i=j
    # Compute derivative of df/da wrt to b
    dadb = -db * a_sum
    dadb += ddetb * sign_u * tf.exp(-log_psi) * w * ddeta_sum  # i=j
    # Compute derivative of df/dw wrt to b
    dwdb = w_dash * -db * k_sum(dw)
    dwdb += w_dash * db / w  # i=j

    return (d2a + dbda + dwda), (d2b + dadb + dwdb), \
           (d2w + dadw + dbdw), \
           None, None, None, None, None, None, None, None, None, None, None, None

# @tf.autograph.experimental.do_not_convert
# @tf.function
@tf.custom_gradient
def first_order_gradient(a_unused, b_unused, w_unused, *fwd_cache):
    """

    :param a_unused:
    :param b_unused:
    :param w_unused:
    :param fwd_cache:
    :return:
    """
    (da, db, dw), first_order_cache = _log_abs_sum_det_first_order(*fwd_cache)
    return (da, db, dw), \
           lambda a_dash, b_dash, w_dash: _log_abs_sum_det_second_order(
               a_dash, b_dash, w_dash, *fwd_cache, *first_order_cache)


# @tf.autograph.experimental.do_not_convert
# @tf.function
@tf.custom_gradient
def log_abs_sum_det(a, b, w):
    """

    :param a:
    :param b:
    :param w:
    :return:
    """
    log_psi, sign, act, sens, fwd_cache = _log_abs_sum_det_fwd(a, b, w)

    def _first_order_grad(dy, dsg, _, __):
        da, db, dw = first_order_gradient(a, b, w, *fwd_cache)

        # print('dy', dy)
        return dy * da, dy * db, tf.reduce_sum(dy * dw, axis=0, keepdims=True)
    # tf.reduce_sum(dy * dw, axis=0, keepdims=True)

    return (log_psi, sign, act, sens), _first_order_grad





In [4]:
n_samples = 100
n_k = 10
ndim = 4
A = tf.random.normal((n_samples, n_k, ndim, ndim), dtype=DTYPE)
B = tf.random.normal((n_samples, n_k, ndim, ndim), dtype=DTYPE)
W = tf.random.normal((1,n_k,1,1), dtype=DTYPE)

A_tc, B_tc, W_tc = convert_to_torch([A, B, W])

with tf.GradientTape(True) as g:
    g.watch(A)
    g.watch(W)
    g.watch(B)
    with tf.GradientTape(True) as gg:
        gg.watch(A)
        gg.watch(B)
        gg.watch(W)
        log_psi,_, _, _ = log_abs_sum_det(A, B, W)
    grad_1_A = gg.gradient(log_psi, A)
    grad_1_B = gg.gradient(log_psi, B)
    grad_1_W = gg.gradient(log_psi, W)
grad_2_A = g.gradient(grad_1_A, A)

log_psi_tc = fn(A_tc, B_tc, W_tc)

# first order
grad_1_Atc = tcgrad(log_psi_tc.sum(), A_tc, retain_graph=True, create_graph=True)[0]
grad_1_Btc = tcgrad(log_psi_tc.sum(), B_tc, retain_graph=True, create_graph=True)[0]
grad_1_Wtc = tcgrad(log_psi_tc.sum(), W_tc, retain_graph=True, create_graph=True)[0]

# second order
grad_2_Atc = tcgrad(grad_1_Atc.sum(), A_tc)[0]

# validation
validate_log_psi = compare_tensors(log_psi_tc, log_psi)
print(validate_log_psi)
validate_gradW = compare_tensors(grad_1_Wtc, grad_1_W)
print(validate_gradW)

validate_grad_1_A = compare_tensors(grad_1_Atc, grad_1_A) 
print(validate_grad_1_A)

validate_grad_2_A = compare_tensors(grad_2_Atc, grad_2_A)
print(validate_grad_2_A)

# if validate_grad1 < 1:
#     log_psi_tc = log_psi_tc.reshape(-1)
#     log_psi = tf.reshape(log_psi, (-1,))
#     for el1, el2 in zip(log_psi_tc, log_psi):
#         print(abs(el1.detach().numpy()-el2.numpy()))
#     print(log_psi_tc, log_psi)
    
#     print('determinants: ', A_tc.det(), B_tc.det())
# print(compare_tensors(grad_1_tc, grad_1))

# print('B: ', ct(B_tc, B))
# print('ae_vectors:', compare_tensors(ae_vectors_tc, ae_vectors))
# print('exponential: ', compare_tensors(expo))

ValueError: ('custom_gradient function expected to return', 14, 'gradients but returned', 15, 'instead.')

In [ ]:
grad_2

In [ ]:
nf_single_in = 4 * n_atoms
nf_single_hidden = 128

nf_hidden_pairwise = 

# params




class fermiNet(tk.Model):
    """
    fermi net, baby
    """
    def __init__(self,
                 r_atoms,
                 n_electrons,
                 n_atoms,
                 n_spin_up,
                 n_spin_down,
                 nf_hidden_single,
                 nf_hidden_pairwise,
                 n_determinants,
                 env_init,
                 full_pairwise,
                 mix_final,
                 mix_input):

        super(fermiNet, self).__init__()

        self.mix_input = mix_input
        self.mix_final = mix_final
        self.full_pairwise = full_pairwise

        # --- initializations
        if full_pairwise:
            n_pairwise = n_electrons ** 2  # - n_electrons
        else:
            n_pairwise = n_electrons ** 2 - n_electrons

        nf_single_in = 4 * n_atoms
        nf_pairwise_in = 4
        nf_single_in_mixed = 3 * nf_single_in + 2 * nf_pairwise_in
        nf_single_intermediate_in = nf_hidden_single * 3 + nf_hidden_pairwise * 2
        nf_pairwise_intermediate_in = nf_hidden_pairwise

        # --- use internally
        self.n_electrons = n_electrons
        self.n_atoms = n_atoms
        self.n_spin_up = n_spin_up
        self.n_spin_down = n_electrons - n_spin_up
        n_spins = n_spin_up + n_spin_down
        self.n_determinants = n_determinants
        self.r_atoms = r_atoms
        self.n_pairwise = n_pairwise

        # --- model
        if self.mix_input:
            self.input_mixer = Mixer(n_electrons, nf_single_in, n_pairwise, nf_pairwise_in, n_spin_up, n_spin_down, full_pairwise, input_mixer=True)
            nf_single_in = nf_single_in_mixed

        self.single_stream_in = Stream(nf_single_in, nf_hidden_single, n_spins, gpu_id, 0)
        self.pairwise_stream_in = Stream(nf_pairwise_in, nf_hidden_pairwise, n_pairwise, gpu_id, 0)
        self.mixer_in = Mixer(n_electrons, nf_hidden_single, n_pairwise, nf_hidden_pairwise, n_spin_up, n_spin_down, full_pairwise)

        self.s1 = Stream(nf_single_intermediate_in, nf_hidden_single, n_spins, gpu_id, 1)
        self.p1 = Stream(nf_pairwise_intermediate_in, nf_hidden_pairwise, n_pairwise, gpu_id, 1)
        self.m1 = Mixer(n_electrons, nf_hidden_single, n_pairwise, nf_hidden_pairwise, n_spin_up, n_spin_down, full_pairwise)

        self.s2 = Stream(nf_single_intermediate_in, nf_hidden_single, n_spins, gpu_id, 2)
        self.p2 = Stream(nf_pairwise_intermediate_in, nf_hidden_pairwise, n_pairwise, gpu_id, 2)
        self.m2 = Mixer(n_electrons, nf_hidden_single, n_pairwise, nf_hidden_pairwise, n_spin_up, n_spin_down, full_pairwise)

        if self.mix_final:
            self.s3 = Stream(nf_single_intermediate_in, nf_hidden_single, n_spins, gpu_id, 3)
            self.p3 = Stream(nf_pairwise_intermediate_in, nf_hidden_pairwise, n_pairwise, gpu_id, 3)
            self.m3 = Mixer(n_electrons, nf_hidden_single, n_pairwise, nf_hidden_pairwise, n_spin_up, n_spin_down, full_pairwise)
            n_envelope_in = nf_single_intermediate_in
        else:
            self.final_single_stream = Stream(nf_single_intermediate_in, nf_hidden_single, n_spins, gpu_id, 3)
            n_envelope_in = nf_hidden_single

        self.envelopes = \
            envelopesLayer(n_spin_up, n_spin_down, n_atoms, n_envelope_in, n_determinants, env_init, gpu_id)

        # self.output_layer = tf.Variable(initializer(n_determinants, (1,n_determinants,1,1), 1, _))
        # self.output_layer = tf.Variable(tf.ones((1, n_determinants, 1, 1))/n_determinants, name='w_1')
        self.output_layer = tf.Variable(env_initializer(16, (1, n_determinants, 1, 1), 1, env_init / n_determinants),
                                        name='wf_1')
        # self.epoch = 1

    # @tf.function  # phase = 0, 1, 2 // test, supervised, unsupervised
    def call(self, r_electrons):
        n_samples = r_electrons.shape[0]
        r_atoms = tf.tile(self.r_atoms, (n_samples, 1, 1))

        # --- computing inputs
        # single_inputs: (b, n_electrons, 4), pairwise_inputs: (, n_pairwise, 4)
        ae_vectors = compute_ae_vectors(r_atoms, r_electrons)
        single, pairwise = compute_inputs(r_electrons, n_samples, ae_vectors, self.n_atoms, self.n_electrons, self.full_pairwise)

        if self.mix_input:
            single = self.input_mixer(single, pairwise, n_samples, self.n_electrons)
        else:
            pass

        # --- input layer
        single, a_in_s, s_in_s = self.single_stream_in(single, n_samples, self.n_electrons)
        pairwise, a_in_p, s_in_p = self.pairwise_stream_in(pairwise, n_samples, self.n_pairwise)
        single_mix = self.mixer_in(single, pairwise, n_samples, self.n_electrons)

        # --- intermediate layers
        tmp, a_1_s, s_1_s = self.s1(single_mix, n_samples, self.n_electrons)
        single += tmp
        tmp, a_1_p, s_1_p = self.p1(pairwise, n_samples, self.n_pairwise)
        pairwise += tmp
        single_mix = self.m1(single, pairwise, n_samples, self.n_electrons)

        tmp, a_2_s, s_2_s = self.s2(single_mix, n_samples, self.n_electrons)
        single += tmp
        tmp, a_2_p, s_2_p = self.p2(pairwise, n_samples, self.n_pairwise)
        pairwise += tmp
        single_mix = self.m2(single, pairwise, n_samples, self.n_electrons)

        # --- final layer
        if self.mix_final:
            tmp, a_3_s, s_3_s = self.s3(single_mix, n_samples, self.n_electrons)
            single += tmp
            tmp, a_3_p, s_3_p = self.p3(pairwise, n_samples, self.n_pairwise)
            pairwise += tmp
            single = self.m3(single, pairwise, n_samples, self.n_electrons)
        else:
            single, a_f, s_f = self.final_single_stream(single_mix, n_samples, self.n_electrons)
            # single += tmp remember to include back in if issues


        # --- envelopes
        spin_up_determinants, spin_down_determinants, a_up, s_up, a_down, s_down = \
            self.envelopes(single, ae_vectors, n_samples)

        # --- logabsdet
        log_psi, sign, a, s = log_abs_sum_det(spin_up_determinants, spin_down_determinants, self.output_layer)

        # yep # 7, 8, 9, 10, 11, 12 #
        if self.mix_final:
            activation = (a_in_s, a_in_p, a_1_s, a_1_p, a_2_s, a_2_p, a_3_s, a_3_p,
                          a_up[0], a_up[1], a_up[2], a_down[0], a_down[1], a_down[2], a)
            sensitivity = (s_in_s, s_in_p, s_1_s, s_1_p, s_2_s, s_2_p, s_3_s, s_3_p,
                           s_up[0], s_up[1], s_up[2], s_down[0], s_down[1], s_down[2], s)
        else:
            activation = (a_in_s, a_in_p, a_1_s, a_1_p, a_2_s, a_2_p, a_f,
                          a_up[0], a_up[1], a_up[2], a_down[0], a_down[1], a_down[2], a)
            sensitivity = (s_in_s, s_in_p, s_1_s, s_1_p, s_2_s, s_2_p, s_f,
                           s_up[0], s_up[1], s_up[2], s_down[0], s_down[1], s_down[2], s)

        return tf.squeeze(log_psi), sign, activation, sensitivity, spin_up_determinants, spin_down_determinants



class envelopesLayer(tk.Model):
    def __init__(self, n_spin_up, n_spin_down, n_atoms, nf_single, n_determinants, env_init, gpu_id):
        super(envelopesLayer, self).__init__()
        # --- variables
        self.n_spin_up = n_spin_up
        self.n_spin_down = n_spin_down
        self.n_k = n_determinants
        self.n_atoms = n_atoms

        # --- envelopes
        self.spin_up_envelope = envelopeLayer(n_spin_up, n_atoms, nf_single, n_determinants, env_init, gpu_id, name='up')
        self.spin_down_envelope = envelopeLayer(n_spin_down, n_atoms, nf_single, n_determinants, env_init, gpu_id, name='down')

    # @tf.function
    def call(self, inputs, ae_vectors, n_samples):
        # --- arrange inputs
        spin_up_ae_vectors, spin_down_ae_vectors = tf.split(ae_vectors, [self.n_spin_up, self.n_spin_down], axis=1)
        spin_up_inputs, spin_down_inputs = tf.split(inputs, [self.n_spin_up, self.n_spin_down], axis=1)

        # --- envelopes
        spin_up_output, a_up, s_up = self.spin_up_envelope(spin_up_inputs, spin_up_ae_vectors,
                                               n_samples, self.n_spin_up, self.n_k, self.n_atoms)
        spin_down_output, a_down, s_down = self.spin_down_envelope(spin_down_inputs, spin_down_ae_vectors,
                                                   n_samples, self.n_spin_down, self.n_k, self.n_atoms)

        return spin_up_output, spin_down_output, a_up, s_up, a_down, s_down


class envelopeLayer(tk.Model):
    def __init__(self, n_spins, n_atoms, nf_single, n_determinants, env_init, gpu_id, name=''):
        super(envelopeLayer, self).__init__()
        # k: n_determinants, i: n_electrons, f: n_features
        w = initializer(nf_single, (n_determinants, n_spins, nf_single, 1), 1, None)
        b = tf.zeros((n_determinants, n_spins, 1, 1))
        w = tf.concat((w, b), axis=2)

        self.w = tf.Variable(w, name='env_%s_w_%i' % (name, n_spins))

        self.Sigma = tf.Variable(env_initializer(3, (n_determinants, n_spins, n_atoms, 3, 3), 3, env_init),
                                 name='env_%s_sigma_%i' % (name, n_spins))

        self.Pi = tf.Variable(env_initializer(n_atoms, (n_determinants, n_spins, n_atoms, 1), 1, env_init),
                              name='env_%s_pi_%i' % (name, n_spins))

    # @tf.function
    def call(self, inputs, ae_vectors, n_samples, n_spins, n_k, n_atoms):
        # inputs: (n_samples, n_electrons, nf_single)
        # ae_vectors: (n_samples, n_electrons, n_atoms, 3)
        # n: n_samples, e: n_electrons, f: nf_single, i: n_electrons, k: n_determinants

        # env_w 'njf,kifs->nkjis'
        inputs_w_bias = tf.concat((inputs, tf.ones((n_samples, n_spins, 1))), axis=-1)
        factor = tf.einsum('njf,kifs->njkis', inputs_w_bias, self.w)

        # k: n_determinants, i: n_electrons, m: n_atoms, n: n_samples, j: n_electrons
        # env_sigma 'njmv,kimvc->nkjimc'
        exponent = tf.einsum('njmv,kimvc->njkimc', ae_vectors, self.Sigma)
        exponential = tf.exp(-tf.norm(exponent, axis=-1))

        # env_pi 'njkim,kims->nkjis'
        exp = tf.einsum('njkim,kims->njkis', exponential, self.Pi)

        output = factor * exp
        output = tf.transpose(output, perm=(0, 2, 3, 1, 4))  # ij ordering doesn't matter / slight numerical diff

        return tf.squeeze(output, -1), (inputs, ae_vectors, exponential), (factor, exponent, exp)


class Stream(tk.Model):
    """
    single / pairwise electron streams
    """
    def __init__(self, in_dim, out_dim, n_spins, gpu_id, node):
        super(Stream, self).__init__()
        # --- variables
        # lim = tf.math.sqrt(6 / (in_dim + out_dim))
        # w = tf.concat((tf.random.uniform((in_dim, out_dim), minval=-lim, maxval=lim), tf.zeros((1, out_dim))), axis=0)
        w = initializer(in_dim, (in_dim, out_dim), out_dim, None)
        b = tf.zeros((1, out_dim))
        # b = tf.random.normal((1, out_dim), stddev=std, dtype=dtype)
        w = tf.concat((w, b), axis=0)
        self.w = tf.Variable(w, name='stream%i_%i' % (node, n_spins))

    def call(self, inputs, n_samples, n_streams):
        inputs_w_bias = tf.concat((inputs, tf.ones((n_samples, n_streams, 1))), axis=-1)
        out1 = inputs_w_bias @ self.w
        out2 = tf.nn.tanh(out1)
        return out2, inputs, out1
        



def fermi_tf(r_atoms, re, w1, w_up, b_up, Sigma_up, Pi_up, w_down, b_down, Sigma_down, Pi_down):
    ae_vectors = compute_ae_vectors(r_atoms, re)
    r2 = tf.einsum('fv,niv->nif',w1,re)

    # Envelopes
    ae_vectors_up = ae_vectors[:,:n_up,...]
    r_up = r2[:,:n_up,:]
    factor = tf.einsum('njf,kfi->nkji', r_up, w_up)
    factor = factor + b_up
    exp = tf.einsum('kimvc,njmv->nkijmc', Sigma_up, ae_vectors_up)
    exponential = tf.exp(-tf.norm(exp, axis=-1))
    exp = tf.einsum('nkijm,kim->nkij', exponential, Pi_up)
    A = factor * exp

    ae_vectors_down = ae_vectors[:,n_up:,...]
    r_down = r2[:,n_up:,:]
    factor = tf.einsum('njf,kfi->nkji', r_down, w_down)
    factor = factor + b_down
    exp = tf.einsum('kimvc,njmv->nkijmc', Sigma_down, ae_vectors_down)
    exponential = tf.exp(-tf.norm(exp, axis=-1))
    exp = tf.einsum('nkijm,kim->nkij', exponential, Pi_down)
    B = factor * exp
    
    return A, B
    
    
def fermi_tc(r_atoms_tc, re_tc, w1_tc,  w_up_tc, b_up_tc, Sigma_up_tc, Pi_up_tc, \
              w_down_tc, b_down_tc, Sigma_down_tc, Pi_down_tc):
    ae_vectors_tc = compute_relative_vectors(re_tc,r_atoms_tc)
    # ### Torch
    r2_tc = tc.einsum('fv,niv->nif',[w1_tc,re_tc])

    # Envelopes
    ae_vectors_up_tc = ae_vectors_tc[:,:n_up,...]
    r_up_tc = r2_tc[:,:n_up,:]
    factor = tc.einsum('njf,kfi->nkji', [r_up_tc, w_up_tc])
    factor = factor + b_up_tc
    exp = tc.einsum('kimvc,njmv->nkijmc', [Sigma_up_tc, ae_vectors_up_tc])
    exponential = tc.exp(-exp.norm(dim=-1))
    exp = tc.einsum('nkijm,kim->nkij', [exponential, Pi_up_tc])
    A_tc = factor * exp

    ae_vectors_down_tc = ae_vectors_tc[:,n_up:,...]
    r_down_tc = r2_tc[:,n_up:,:]
    factor = tc.einsum('njf,kfi->nkji', [r_down_tc, w_down_tc])
    factor = factor + b_down_tc
    exp = tc.einsum('kimvc,njmv->nkijmc', [Sigma_down_tc, ae_vectors_down_tc])
    exponential = tc.exp(-exp.norm(dim=-1))
    exp = tc.einsum('nkijm,kim->nkij', [exponential, Pi_down_tc])
    B_tc = factor * exp
    return A_tc, B_tc
    

In [ ]:
n_samples = 100
n_k = 10
n_f = 20
n_up = 2
n_e = 4
n_down = n_e - n_up
n_atom = 1 

### Parameters
# First
r_atoms = tf.zeros((n_samples, n_atom, 3), dtype=DTYPE)
re = tf.random.normal((n_samples, n_e, 3), dtype=DTYPE) #* 100.
w1 = tf.random.normal((n_f, 3), dtype=DTYPE)
# up
w_up = tf.random.normal((n_k, n_f, n_up), dtype=DTYPE)
b_up = tf.random.normal((n_k, n_up, 1), dtype=DTYPE)
Sigma_up = tf.random.normal((n_k, n_up, n_atom, 3, 3), dtype=DTYPE)
Pi_up = tf.random.normal((n_k, n_up, n_atom), dtype=DTYPE)
# down
w_down = tf.random.normal((n_k, n_f, n_down), dtype=DTYPE)
b_down = tf.random.normal((n_k, n_down, 1), dtype=DTYPE)
Sigma_down = tf.random.normal((n_k, n_down, n_atom, 3, 3), dtype=DTYPE)
Pi_down = tf.random.normal((n_k, n_down, n_atom), dtype=DTYPE)
# Weights
W = tf.random.normal((1, n_k, 1, 1), dtype=DTYPE)

with tf.GradientTape(True) as g:
    g.watch(W)
    g.watch(re)
    with tf.GradientTape(True) as gg:
        gg.watch(re)
        gg.watch(W)
        A, B = fermi_tf(r_atoms, re, w1,  w_up, b_up, Sigma_up, Pi_up, \
                         w_down, b_down, Sigma_down, Pi_down)
        log_psi,_ = log_abs_sum_det(A, B, W)

    grad_1_A = gg.gradient(log_psi, A)
    grad_1_B = gg.gradient(log_psi, B)
    grad_1_W = gg.gradient(log_psi, W)
    grad_1_re = gg.gradient(log_psi, re)
    grads_1_re = tf.reshape(grad_1_re, (-1, n_e*3))
    grads_1_re = [grads_1_re[..., i] for i in range(grads_1_re.shape[-1])]
    
grad_2_re = g.gradient(grads_1_re[0], re)
grad_2_A = g.gradient(grad_1_A, A)

# print(grad_2_re)
# print(grad_2)

### Torch
re_tc, w1_tc, w_up_tc, b_up_tc, w_down_tc, b_down_tc, W_tc = \
convert_to_torch([re, w1, w_up, b_up, w_down, b_down, W])
Sigma_up_tc, Pi_up_tc, Sigma_down_tc, Pi_down_tc = \
convert_to_torch([Sigma_up, Pi_up, Sigma_down, Pi_down])
r_atoms_tc = convert_to_torch([r_atoms])[0]


A_tc, B_tc = fermi_tc(r_atoms_tc, re_tc, w1_tc, w_up_tc, b_up_tc, Sigma_up_tc, Pi_up_tc, \
              w_down_tc, b_down_tc, Sigma_down_tc, Pi_down_tc)
log_psi_tc = fn(A_tc, B_tc, W_tc)

grad_1_Atc = tcgrad(log_psi_tc.sum(), A_tc, retain_graph=True, create_graph=True)[0]
grad_2_Atc = tcgrad(grad_1_Atc.sum(), A_tc, retain_graph=True)[0]
grad_1_Btc = tcgrad(log_psi_tc.sum(), B_tc, retain_graph=True)[0]
grad_1_Wtc = tcgrad(log_psi_tc.sum(), W_tc, retain_graph=True)[0]
grad_1_retc = tcgrad(log_psi_tc.sum(), re_tc, retain_graph=True, create_graph=True)[0]
grads_1_retc = grad_1_retc.view(-1,3*n_e)
grad_2_retc = tcgrad(grads_1_retc[:,0].sum(), re_tc, retain_graph=True)[0]

grad_1 = tcgrad(log_psi_tc.sum(), A_tc)

validate_log_psi = compare_tensors(log_psi_tc, log_psi)
print(validate_log_psi)
validate_gradW = compare_tensors(grad_1_Wtc, grad_1_W)
print(validate_gradW)

print(compare_tensors(grad_1_Atc, grad_1_A))
print(compare_tensors(grad_1_Btc, grad_1_B))

validate_re = ct(grad_1_retc, grad_1_re)
print('re grad: ', validate_re)

validate_grads_re = ct(grads_1_retc[:,0], grads_1_re[0])
print('re grads: ', validate_grads_re)

validate_2_re = ct(grad_2_retc, grad_2_re)
print('re grad: ', validate_2_re)

validate_2_A = ct(grad_2_Atc, grad_2_A)
print('re grad: ', validate_2_A)



In [ ]:
grad_1_retc.shape

In [ ]:
grad_1_re_loss

In [ ]:
grad_2_A

In [ ]:
grad_2_retc

In [ ]:
# --- compare nico to new 

# --- outputs the same

# --- grad1 the same (ABW)

# --- grad1 the same (r)

# --- grad2 the same (ABW)

# --- grad2 the same (r)

# --- speed